   # Within-fire variability 
   
   Workflow to extract pixels from imagery and boostrap samples to get median and mean for each image.

In [ ]:
import xarray as xr
import geopandas as gpd
import pandas as pd
import numpy as np
import dask
%matplotlib inline
%tb


In [391]:
# required functions
%run processing_functions.py
%run ../pixel_to_df.py

## Test workflow on single fire

What is the distribution of pixel counts in  defol/non-defoliated between and within fires?


Goals: 
1) create reference distribution from one of defoliation / non-defoiated halves
2) from the opposite half, subsample to match sample size of reference. 
3) boostrap resample to get CI and distrbution of values. 
4) compare distribution

Steps:
1) get coordinates and return them as a dataframe with x and y
2) add column to df which says whether it is defoliated or non-defoliated (binary)
3) extract pixel values? 
4) randomly select a subsample of pixels from df, get values...etc.?

* turn into 4d array with time, band, x, y - band would be nbr and defoliated (1,0)


NOTES:
- values to be standardized when in DF format after pixel extraction

In [134]:
# read in data and get coordinates
ds = rx.open_rasterio("/Users/jgoldman/Desktop/chp3/clipped/nbr/non-defoliated/COC16_1997_1038.nc", masked=True)


In [392]:
# random sample for a given year.
# for each sample take the coordinates and get values across df.
def random_sample(da,
                  n, 
                  dim_slice,
                  keep_cols=True):
    """
    take a data array, turn to df and get random sample
    da = xarray data array with n dimensions
    n = number of samples
    dim_slice = slice along certain time dimension
    keep_cols = 

    """
    # slice along dimension
    da2= da.isel(time=dim_slice)
    #to dataframe
    df = da2.to_dataframe()

    # sample location
    samples = []
    no_of_points = n
    # random sample of n points
    sample_loc = df.dropna().sample(n=int(round(no_of_points)))
    samples.append(sample_loc)
    #join back into single datafame
    all_samples = pd.concat([samples[i] for i in range(0,len(samples))])
    
    # x and y to lat and lon column
    all_samples.reset_index(inplace = True)
    all_samples = all_samples.rename(columns={'y': 'Lat', 'x': 'Long'}) 
    
    if keep_cols==False:
        #remove crs, nbr and time cols
        all_samples = all_samples.drop(columns =['crs', 'time', 'nbr'])
     # for each lat/long select pixel
    pix_samples = []
    for index, row in all_samples.iterrows():
        x = row['Long']
        y = row['Lat']
        df2 = da.to_dataframe()
        df2 = df2.dropna()
        df2 = std_nbr(df2)
        df2 = df2.loc[(df2['y'] == y) & (df2['x'] == x)]
        # add add row that goes preNBR - nbr 10
        ints = np.arange(0,12,1)
        df2['nbr_time'] = ints
        df2['nbr_time'] = df2['nbr_time'].replace({'0': 'preNBR', '1': 'nbrYOF', '2': '1', '3': '2', '4': '3', '5': '4', '6': '5', '7': '6', '8': '7', '9':'8', '10': '9', '11': '10'})
        pix_samples.append(df2)

    #join back into single datafame
    all_samples = pd.concat([pix_samples[i] for i in range(0,len(pix_samples))])
  
    #remove crs
    all_samples = all_samples.drop(columns =['crs'])
    
    ids = np.arange(1,1001,1)
    # add pixel id
    all_samples['id'] = np.repeat(ids, 12, axis = 0)
    
    return(all_samples)

In [393]:
df = random_sample(ds, 1000, 1, False)
df



,year,y,x,nbr,nbr_time,id
27998,1996,49.868,-80.504,0.459,0,1
58180,1997,49.868,-80.504,0.454,1,1
88362,1998,49.868,-80.504,0.448,2,1
118544,1999,49.868,-80.504,0.443,3,1
148726,2000,49.868,-80.504,0.437,4,1
...,...,...,...,...,...,...
237425,2003,49.872,-80.580,0.291,7,1000
267607,2004,49.872,-80.580,0.282,8,1000
297789,2005,49.872,-80.580,0.274,9,1000
327971,2006,49.872,-80.580,0.286,10,1000


In [363]:
def magnitude_calc(x):
    pre= x[x['nbr_time'] == 0]
    pre = pre['nbr'].item()
    post= x[x['nbr_time'] == 1]
    post = post['nbr'].item()
    p_diff = pre - post
    rec = x[x['nbr_time'] == 11]
    rec = rec['nbr'].item()
    rec_diff = rec - post
    mag = (rec - post)* 100
    
    return pd.DataFrame({'recovery_magnitude': [mag]})



In [364]:
def recovery_magnitude(df):
    grp = df.groupby('id')
    magnitudes = []
    # for length in 1-1001
    ids = np.arange(1,1001,1)
    for i in ids:
        # get group
        x = grp.get_group(i)
        # calc mac
        res = magnitude_calc(x)
        # add id column
        res['id'] = i
        # append to list
        magnitudes.append(res)
        
        
    #join back into single datafame
    all_mags = pd.concat([magnitudes[i] for i in range(0,len(magnitudes))])
    
    return(all_mags)


In [394]:
recovery_magnitude(df)

,recovery_magnitude,id
0,-1.945,1
0,29.986,2
0,20.033,3
0,6.551,4
0,22.127,5
...,...,...
0,21.128,996
0,35.396,997
0,51.978,998
0,45.702,999


In [399]:
ds2 = ds.isel(time= 0)
ds2 = ds2.to_dataframe().reset_index()
ds2.dropna()

,y,x,time,crs,nbr
152,49.944,-80.559,1996.000,0,0.525
153,49.944,-80.558,1996.000,0,0.525
560,49.943,-80.559,1996.000,0,0.342
561,49.943,-80.559,1996.000,0,0.342
563,49.943,-80.559,1996.000,0,0.423
...,...,...,...,...,...
136559,49.854,-80.571,1996.000,0,0.555
136560,49.854,-80.570,1996.000,0,0.523
136969,49.854,-80.571,1996.000,0,0.588
136970,49.854,-80.571,1996.000,0,0.588


NEED TO MAKE FUNCTION THAT :

calculates the distribution for the reference portion of the fire.

steps:
1. slice og dataframe by dimension time = 1 using isel. This returns all the pixel locations for a given year that we then use to iterate throught the og dataframe
2. drop NA
3. iterate through the OG dataframe

In [405]:
def magnitude_calc_prep(da):
    """
    calculates the distribution for the reference portion of the fire.
    Steps:
    1. slice og dataframe by dimension time = 1 using isel. 
        This returns all the pixel locations for a given year that we then 
        use to iterate throught the og dataframe
    2. drop NA
    3. iterate through the OG dataframe
    
    """
    da2 = da.isel(time= 0)
    da2 = da.to_dataframe().reset_index()
    da2.dropna()
    
    pix_samples = []
    for index, row in da2.iterrows():
        x = row['x']
        y = row['y']
        df2 = da.to_dataframe()
        df2 = df2.dropna()
        df2 = std_nbr(df2)
        df2 = df2.loc[(df2['y'] == y) & (df2['x'] == x)]
        # add add row that goes preNBR - nbr 10
        ints = np.arange(0,12,1)
        df2['nbr_time'] = ints
        df2['nbr_time'] = df2['nbr_time'].replace({'0': 'preNBR', '1': 'nbrYOF', '2': '1', '3': '2', '4': '3', '5': '4', '6': '5', '7': '6', '8': '7', '9':'8', '10': '9', '11': '10'})
        pix_samples.append(df2)
    
    #join back into single datafame
    all_samples = pd.concat([pix_samples[i] for i in range(0,len(pix_samples))])
  
    #remove crs
    all_samples = all_samples.drop(columns =['crs'])
    
    # set ids, but need to now pixel_count so get length of da2
    pix_count = len(da2)
    ids = np.arange(1,pix_count,1)
    # add pixel id
    all_samples['id'] = np.repeat(ids, 12, axis = 0)
    
    return(all_samples)


KeyboardInterrupt: 